In [ ]:
#general purpose distmat with cluster coloring and sorting
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.sampledata.les_mis import data
from colour import colour
from sklearn.cluster import DBSCAN

def bokeh_distmat(labels, distmat, plot_title ='distmat' , similaritymat = False):
    
    db = DBSCAN(eps=0.3, min_samples=10,metric='precomputed',n_jobs=-1).fit(distmat)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    clustLabels = db.labels_
    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(clustLabels)) 
    
    
    
    
    D=1/D
    np.fill_diagonal(D,0)
    
    Y = sch.linkage(D, method='centroid')
    Z1 = sch.dendrogram(Y, orientation='right')
    
    # Plot distance matrix.
    idx1 = Z1['leaves']
    idx2 = Z2['leaves']
    D = D[idx1,:]
    D = D[:,idx1]
    
    
    
    grey = Color("grey")
    red = Color("red")
    blue = Color("blue")
    clusterColors = list(red.range_to(blue, nclusters))
    clusterColors.append(grey)
    N = len(labels)
    colormap = [ str(c.hex) for c in clusterColors]
    xname = labels
    yname = labels
    color = []
    alpha = []
    
    for i, node1 in enumerate(labels):
        for j, node2 in enumerate(labels):
            xname.append(node1)
            yname.append(node2)
            if similaritymat == True:
                alpha.append(min(distmat[i,j]/np.amax(distmat), 0.9) + 0.1)
            else:
                alpha.append(min(np.amin(distmat)/distmat[i,j], 0.9) + 0.1)
                
            if clustLabels[i] == clustLabels[j]:
                color.append(colormap[clustLabels[i]])
            else:
                color.append(colormap[-1])

    source = ColumnDataSource(data=dict(
        xname=xname,
        yname=yname,
        colors=color,
        alphas=alpha,
        count=counts.flatten(),
    ))

    p = figure(title= plot_title,
               x_axis_location="above", tools="hover,save",
               x_range=list(reversed(names)), y_range=names)

    p.plot_width = 800
    p.plot_height = 800
    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "5pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = np.pi/3

    p.rect('xname', 'yname', 0.9, 0.9, source=source,
           color='colors', alpha='alphas', line_color=None,
           hover_line_color='black', hover_color='colors')

    p.select_one(HoverTool).tooltips = [
        ('names', '@yname, @xname'),
        ('count', '@count'),
    ]

    output_notebook()
    show(p) # show the plot

In [ ]:
from bokeh.layouts import row, column
from bokeh.models import BoxSelectTool, LassoSelectTool, Spacer
from bokeh.plotting import figure, curdoc

def Bokeh_scatter(Xdata,Ydata):

    x = Xdata.flatten()
    y = Ydata.flatten()

    TOOLS="pan,wheel_zoom,box_select,lasso_select,reset"

    # create the scatter plot
    p = figure(tools=TOOLS, plot_width=600, plot_height=600, min_border=10, min_border_left=50,
               toolbar_location="above", x_axis_location=None, y_axis_location=None,
               title="Linked Histograms")
    p.background_fill_color = "#fafafa"
    p.select(BoxSelectTool).select_every_mousemove = False
    p.select(LassoSelectTool).select_every_mousemove = False

    r = p.scatter(x, y, size=3, color="#3A5785", alpha=0.6)

    # create the horizontal histogram
    hhist, hedges = np.histogram(x, bins=20)
    hzeros = np.zeros(len(hedges)-1)
    hmax = max(hhist)*1.1

    LINE_ARGS = dict(color="#3A5785", line_color=None)

    ph = figure(toolbar_location=None, plot_width=p.plot_width, plot_height=200, x_range=p.x_range,
                y_range=(-hmax, hmax), min_border=10, min_border_left=50, y_axis_location="right")
    ph.xgrid.grid_line_color = None
    ph.yaxis.major_label_orientation = np.pi/4
    ph.background_fill_color = "#fafafa"

    ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hhist, color="white", line_color="#3A5785")
    hh1 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.5, **LINE_ARGS)
    hh2 = ph.quad(bottom=0, left=hedges[:-1], right=hedges[1:], top=hzeros, alpha=0.1, **LINE_ARGS)

    # create the vertical histogram
    vhist, vedges = np.histogram(y, bins=20)
    vzeros = np.zeros(len(vedges)-1)
    vmax = max(vhist)*1.1

    pv = figure(toolbar_location=None, plot_width=200, plot_height=p.plot_height, x_range=(-vmax, vmax),
                y_range=p.y_range, min_border=10, y_axis_location="right")
    pv.ygrid.grid_line_color = None
    pv.xaxis.major_label_orientation = np.pi/4
    pv.background_fill_color = "#fafafa"

    pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vhist, color="white", line_color="#3A5785")
    vh1 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.5, **LINE_ARGS)
    vh2 = pv.quad(left=0, bottom=vedges[:-1], top=vedges[1:], right=vzeros, alpha=0.1, **LINE_ARGS)

    layout = column(row(p, pv), row(ph, Spacer(width=200, height=200)))

    curdoc().add_root(layout)
    curdoc().title = "Selection Histogram"
    
    output_notebook()
    show(p) # show the plot

def update(attr, old, new):
    inds = np.array(new['1d']['indices'])
    if len(inds) == 0 or len(inds) == len(x):
        hhist1, hhist2 = hzeros, hzeros
        vhist1, vhist2 = vzeros, vzeros
    else:
        neg_inds = np.ones_like(x, dtype=np.bool)
        neg_inds[inds] = False
        hhist1, _ = np.histogram(x[inds], bins=hedges)
        vhist1, _ = np.histogram(y[inds], bins=vedges)
        hhist2, _ = np.histogram(x[neg_inds], bins=hedges)
        vhist2, _ = np.histogram(y[neg_inds], bins=vedges)
    hh1.data_source.data["top"]   =  hhist1
    hh2.data_source.data["top"]   = -hhist2
    vh1.data_source.data["right"] =  vhist1
    vh2.data_source.data["right"] = -vhist2

r.data_source.on_change('selected', update)

In [ ]:
#BOKEH SCATTER -> adapt to show taxonomic nodes and HOGname on hover 

import numpy as np

from bokeh.plotting import figure, show, output_file
def scatter_labels( df , col1, col2, labelcol):
    TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"
    p = figure(tools=TOOLS)
    
    source = ColumnDataSource(data=dict(
        x=x,
        y=y,
        desc=df[],
    ))

    p.scatter(x, y, radius=radii,
              fill_color=colors, fill_alpha=0.6,
              line_color=None )

    output_file("color_scatter.html", title="color_scatter.py example")
    output_notebook()
    show(p)  

In [ ]:

p = Scatter(df, x='mpg', y='hp', title="HP vs MPG", color="navy",
            xlabel="Miles Per Gallon", ylabel="Horsepower")

output_file("scatter.html")